# nf_loto_platform 運用確認ノートブック
（ops / 検証 / ログ・生成物チェック用）

このノートブックは、`nf_loto_platform` 一式がローカル環境で正しく動作しているかを、
**運用視点・機能視点・テストケース視点・生成物／ログ視点**から段階的に確認するためのものです。

## このノートブックでできること

- DB 接続／メタデータテーブルの存在確認・初期化
- 特徴量生成パイプラインのスモークテスト
- AutoML / モデル実行バックエンドの最小実行
- ログ・メトリクス・HTML レポートなどの生成物確認
- pytest を使った回帰テストの一部実行

**重要:**  
このノートブックは *設計上* すべてのセルを順番に実行できるようにしてありますが、
実際の DB 接続先や GPU/ライブラリなどの環境依存があるため、
必要に応じてセルをスキップ・編集しながら使ってください。

## 0. 環境セットアップ

まず、プロジェクトルートと `src` を Python パスに通し、バージョン情報を確認します。

In [ ]:
from pathlib import Path
import sys
import os

# プロジェクトルート（このノートブックが置かれている notebooks/ の 1 つ上）
NOTEBOOK_PATH = Path.cwd()
PROJECT_ROOT = NOTEBOOK_PATH.parent
SRC_ROOT = PROJECT_ROOT / "src"

print("NOTEBOOK_PATH:", NOTEBOOK_PATH)
print("PROJECT_ROOT :", PROJECT_ROOT)
print("SRC_ROOT     :", SRC_ROOT)

if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))
    print("=> sys.path に SRC_ROOT を追加しました")

# バージョン情報など（必要に応じて追加）
import platform
print("Python :", sys.version)
print("Platform:", platform.platform())

try:
    import nf_loto_platform
    print("nf_loto_platform version: package import OK")
except Exception as e:  # noqa: BLE001
    print("nf_loto_platform import error:", e)


## A. DB・テーブルの確認／初期化／セットアップ

このセクションでは、以下を順番に行います。

1. DB 設定ファイル（`config/db.yaml` or `db.yaml.template`）の確認
2. `nf_loto_platform.db.postgres_manager` を使った接続テスト
3. `sql/001_create_nf_model_run_tables.sql` などの DDL の確認
4. 必要に応じて `setup_postgres` を使ったテーブル初期化

実際の接続情報は、あなたのローカル環境の設定に合わせて編集してください。

### A-1. DB 設定ファイルの読み込みと確認

In [ ]:
from nf_loto_platform.core import settings

db_cfg = settings.load_db_config()
print("DB 設定:", db_cfg)

if not db_cfg:
    print("""[注意] config/db.yaml または db.yaml.template から有効な設定が読み込めていません。
      このノートブックを進める前に、DB 設定を更新してください。
""")


### A-2. PostgreSQL 接続テスト

In [ ]:
from nf_loto_platform.db import postgres_manager

conn = None
try:
    conn = postgres_manager.get_connection()
    print("接続成功:", conn)
    with conn.cursor() as cur:
        cur.execute("SELECT version();")
        ver = cur.fetchone()
        print("PostgreSQL version:", ver)
except Exception as e:  # noqa: BLE001
    print("[接続エラー] DB に接続できませんでした:", e)
finally:
    if conn is not None:
        conn.close()


### A-3. nf_* メタデータテーブルの存在確認

In [ ]:
from nf_loto_platform.db_metadata import schema_definitions

NF_TABLES = [
    schema_definitions.NF_MODEL_RUNS_TABLE,
    schema_definitions.NF_MODEL_REGISTRY_TABLE,
    schema_definitions.NF_FEATURE_SETS_TABLE,
    schema_definitions.NF_DATASETS_TABLE,
    schema_definitions.NF_DRIFT_METRICS_TABLE,
    schema_definitions.NF_RESIDUAL_ANOMALIES_TABLE,
    schema_definitions.NF_REPORTS_TABLE,
]

print("チェック対象テーブル:", NF_TABLES)

def list_tables():
    from nf_loto_platform.db import postgres_manager
    with postgres_manager.get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT tablename
                FROM pg_catalog.pg_tables
                WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
                ORDER BY tablename;
            """)
            return [r[0] for r in cur.fetchall()]

try:
    existing = list_tables()
    print("既存テーブル一覧:", existing)
    missing = [t for t in NF_TABLES if t not in existing]
    if missing:
        print("[注意] 足りないテーブル:", missing)
    else:
        print("すべてのメタデータテーブルが存在します。")
except Exception as e:  # noqa: BLE001
    print("[エラー] テーブル一覧取得に失敗:", e)


### A-4. DDL の実行によるメタデータテーブル初期化

必要であれば、`nf_loto_platform.db.setup_postgres` を経由して
`sql/001_create_nf_model_run_tables.sql` などの DDL を実行し、メタデータテーブルを作成します。

> ⚠ **注意:** 本番環境で実行する場合は、必ず事前にバックアップを取得してから実行してください。

In [ ]:
from nf_loto_platform.db import setup_postgres

# 実行前に本当に良いかどうかのフラグ
EXECUTE_DDL = False  # True にすると DDL を実行

if EXECUTE_DDL:
    try:
        setup_postgres.create_metadata_tables()
        print("メタデータテーブルの作成を完了しました。")
    except Exception as e:  # noqa: BLE001
        print("[エラー] DDL 実行に失敗:", e)
else:
    print("EXECUTE_DDL=False のため、DDL は実行していません。")
    print("DDL を実行したい場合は、このセル内のフラグを True に変更してください。")


## B. 特徴量生成パイプラインの確認

このセクションでは、ロト系のベーステーブルから特徴量テーブルを作るまでの流れを確認します。

- B-1. ベーステーブル（例: `nf_loto_final`）の行数・カラム確認
- B-2. `loto_etl` / `loto_etl_updated` による前処理スモークテスト
- B-3. 特徴量テーブル（futr/hist/stat_*）のスキーマ確認

※ 実際にどのテーブルから特徴量を生成するかは、あなたの DB のスキーマに合わせて変更してください。

### B-1. ベーステーブルの確認

In [ ]:
import pandas as pd
from nf_loto_platform.db import postgres_manager

BASE_TABLE = "nf_loto_final"  # 必要に応じて変更

try:
    with postgres_manager.get_connection() as conn:
        q = f"SELECT * FROM {BASE_TABLE} ORDER BY 1 LIMIT 100;"
        df_base = pd.read_sql(q, conn)
    print("ベーステーブル:", BASE_TABLE)
    print("shape:", df_base.shape)
    display(df_base.head())
except Exception as e:  # noqa: BLE001
    print("[エラー] ベーステーブル読み込みに失敗:", e)
    df_base = None


### B-2. ETL（loto_etl / loto_etl_updated）のスモークテスト

In [ ]:
from nf_loto_platform.db import loto_etl, loto_etl_updated

# ここでは「関数が呼べるか／最低限の戻り値が得られるか」を確認するイメージです。
# 実際のシグネチャは実装に合わせて編集してください。

try:
    print("loto_etl モジュール:", loto_etl)
    print("loto_etl_updated モジュール:", loto_etl_updated)
    # 例:
    # etl_df = loto_etl.run_etl(limit=1000)
    # print("etl_df shape:", etl_df.shape)
except Exception as e:  # noqa: BLE001
    print("[エラー] ETL スモークテストに失敗:", e)


### B-3. 特徴量テーブルのスキーマ確認

In [ ]:
# 特徴量テーブル名は、あなたの環境の命名に合わせて変更してください。
FEATURE_TABLE = "nf_loto_features"  # 仮の例

try:
    from nf_loto_platform.db import postgres_manager
    import pandas as pd

    with postgres_manager.get_connection() as conn:
        q = f"SELECT * FROM {FEATURE_TABLE} ORDER BY 1 LIMIT 100;"
        df_feat = pd.read_sql(q, conn)
    print("特徴量テーブル:", FEATURE_TABLE)
    print("shape:", df_feat.shape)
    print("カラム一覧:", df_feat.columns.tolist())
    display(df_feat.head())
except Exception as e:  # noqa: BLE001
    print("[注意] 特徴量テーブルの読み込みに失敗:", e)
    print("まだ特徴量生成が未実行、またはテーブル名が環境と合っていない可能性があります。")


## C. AutoML / モデル実行バックエンドの確認

このセクションでは、NeuralForecast AutoModels ベースの学習パイプラインが
最低限動作するかどうかを確認します。

- C-1. AutoModelBuilder のインポートと登録モデル一覧の確認
- C-2. 極小データセットでのテスト実行（オプション）

### C-1. AutoModelBuilder の確認

In [ ]:
from nf_loto_platform.ml import automodel_builder

print("AutoModelBuilder モジュール:", automodel_builder)

# 登録済みモデル一覧を出すヘルパーがあれば利用し、無ければ dir() ベースで確認
candidates = [name for name in dir(automodel_builder) if "Auto" in name or "create" in name.lower()]
print("AutoModel 関連と思しきシンボル:", candidates)


### C-2. 極小データセットでのテスト実行（オプション）

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

from nf_loto_platform.ml import model_runner

# 極小サンプルデータセットを構築（NeuralForecast 用の基本形式）
dates = [datetime(2020, 1, 1) + timedelta(days=i) for i in range(30)]
df_small = pd.DataFrame({
    "unique_id": ["series_1"] * len(dates),
    "ds": dates,
    "y": [i % 10 for i in range(30)],
})

print("サンプルデータ:")
display(df_small.head())

# 実際の model_runner の API に合わせて設定を組む必要があります。
# ここでは疑似コード／テンプレ的な例として示します。
RUN_TINY_EXPERIMENT = False  # True にすると実際に学習を試みる

if RUN_TINY_EXPERIMENT:
    try:
        result = model_runner.run_single_model_experiment(
            data=df_small,
            model_name="AutoNHITS",   # 実際にインストールされている Auto* 名に合わせて変更
            horizon=7,
            max_trials=1,
            timeout_s=60,
        )
        print("実験結果:", result)
    except Exception as e:  # noqa: BLE001
        print("[エラー] 極小実験の実行に失敗:", e)
else:
    print("RUN_TINY_EXPERIMENT=False のため、実際の学習は実行していません。")


## D. ログ・モニタリング・リソース利用の確認

ここでは、MLflow / W&B ロガー、および Prometheus / resource_monitor など
モニタリング系のコンポーネントを軽く叩いてみます。

### D-1. MLflow / W&B ロガーのインポート確認

In [ ]:
from nf_loto_platform.logging_ext import mlflow_logger, wandb_logger

print("mlflow_logger モジュール:", mlflow_logger)
print("wandb_logger  モジュール:", wandb_logger)


### D-2. リソース利用スナップショットの取得

In [ ]:
from nf_loto_platform.monitoring import resource_monitor

info = resource_monitor.collect_resource_usage()
print("resource_monitor.collect_resource_usage() ->")
for k, v in info.items():
    print(f"  {k}: {v}")


## E. WebUI / Streamlit 連携の確認

Streamlit WebUI 自体はノートブック内では直接起動しませんが、
少なくとも Python から import できるか／エントリポイントが見つかるかを確認します。

In [ ]:
from pathlib import Path

streamlit_app_path = PROJECT_ROOT / "apps" / "webui_streamlit" / "streamlit_app.py"
runner_path = PROJECT_ROOT / "apps" / "webui_streamlit" / "nf_auto_runner_full.py"

print("streamlit_app.py :", streamlit_app_path, "exists:", streamlit_app_path.exists())
print("nf_auto_runner_full.py:", runner_path, "exists:", runner_path.exists())

print("""ローカルのシェルからは、例えば以下のように起動できます:
  streamlit run apps/webui_streamlit/streamlit_app.py
""")


## F. 生成物・ログ・メタデータの確認

このセクションでは、モデル実行後に蓄積されることを想定しているテーブルや
レポートファイルなどを確認します。

- F-1. `nf_model_runs` / `nf_model_metrics` などのメタデータテーブル
- F-2. HTML レポートの例


### F-1. nf_* メタデータテーブル中身の確認

In [ ]:
import pandas as pd
from nf_loto_platform.db_metadata import schema_definitions
from nf_loto_platform.db import postgres_manager

tables_to_inspect = [
    schema_definitions.NF_MODEL_RUNS_TABLE,
    schema_definitions.NF_MODEL_METRICS_TABLE,
    schema_definitions.NF_DRIFT_METRICS_TABLE,
]

for tbl in tables_to_inspect:
    try:
        with postgres_manager.get_connection() as conn:
            q = f"SELECT * FROM {tbl} ORDER BY 1 LIMIT 20;"
            df = pd.read_sql(q, conn)
        print(f"テーブル {tbl}: shape={df.shape}")
        display(df.head())
    except Exception as e:  # noqa: BLE001
        print(f"[注意] テーブル {tbl} の読み込みに失敗:", e)


### F-2. HTML レポートの表示例

In [ ]:
from nf_loto_platform.reports import html_reporter

# シンプルなレポート文字列を作成
html = html_reporter.render_simple_report(title="nf_loto_platform レポート例", body="これはテスト用のレポートです。")

# 一時ファイルに書き出し
from pathlib import Path
tmp_report = PROJECT_ROOT / "tmp_nf_loto_report_example.html"
html_reporter.write_report(tmp_report, html)
print("HTML レポートを書き出しました:", tmp_report)

# Jupyter 上で簡易表示（本格的なレンダリングはブラウザでファイルを開いて確認）
from IPython.display import HTML
HTML(html)


## G. pytest による回帰テストの一部実行

最後に、ノートブックから pytest を呼び出して、
一部または全テストを実行するサンプルを示します。

> ⚠ 注意:  
> フルテストは時間がかかる場合があります。まずは静的テストや core 層など、スコープを絞って実行することを推奨します。

In [ ]:
import subprocess
import sys

def run_pytest(args):
    cmd = [sys.executable, "-m", "pytest"] + args
    print("実行コマンド:", " ".join(cmd))
    result = subprocess.run(cmd, cwd=PROJECT_ROOT, text=True)
    return result.returncode

# 例1: 静的チェックのみ
# rc = run_pytest(["tests/static"])

# 例2: core / db / ml のみ
# rc = run_pytest(["tests/core", "tests/db", "tests/ml"])

print("必要なスコープを選んでコメントアウトを外し、実行してください。")
